In [1]:
import json
from twarc_csv import DataFrameConverter
from twarc.expansions import ensure_flattened
import pandas as pd
from twarc_csv import dataframe_converter
from twarc_csv import CSVConverter, DataFrameConverter
import math
import emoji
from operator import itemgetter
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from advertools import extract_emoji
import numpy as np
import pickle

**Manual Input**: Set the path.

In [2]:
path='C:/Users/acoub/OneDrive/Desktop/DSDM/Thesis/ChileGov/methodology_feminism'
sep='/'

In [3]:
path_scripts = path+sep+'initial'
path_data= path+sep+'data'

**Manual Input**: List of politicians usernames to search the most popular hashtags.  
Alternative: Paste in the data folder one txt file named "Left_Politicians.txt" with one username per line for left politicians and the same for right politicians in a file "Right_Politicians.txt" 

In [ ]:
left_wing=['rightusername1','rightusername2','rightusername3']
right_wing=['leftusername1','leftusername2','leftusername3']

If you added directly the txt file to the folder data, don't run the following cell.

In [ ]:
with open(path_data+sep+'Left_Politicians.txt', 'w') as f:
    for line in left_wing:
        f.write(line)
        f.write('\n')
        
with open(path_data+sep+'Right_Politicians.txt', 'w') as f:
    for line in right_wing:
        f.write(line)
        f.write('\n')

**Manual input:** Define the period of time when you want to evaluate the use of hashtags to label Left and Right. We recomend a period when is easy to identificate right wing and left wing people (for instance an electoral period). Format YYYY-MM-DD

In [4]:
start_date="2021-10-19"
end_date="2021-12-20"

In [5]:
filename = path_data+sep+'start_date_for_label.sav'
pickle.dump((start_date), open(filename, 'wb'))

filename = path_data+sep+'end_date_for_label.sav'
pickle.dump((end_date), open(filename, 'wb'))

**Download the tweets**

In [ ]:
file=path_data+sep+'Left_Politicians.txt'
%cd {path_data}
! twarc2 timelines --archive --start-time {start_date} --end-time {end_date} {file} left_hashtags.json

In [ ]:
file=path_data+sep+'Right_Politicians.txt'
%cd {path_data}
! twarc2 timelines --archive --start-time {start_date} --end-time {end_date} {file} right_hashtags.json

Run the following cells only after you download the tweets.

In [6]:
converter = DataFrameConverter()
data = []
with open(path_data+sep+"left_hashtags.json") as f:
    for line in f:
        data.append(json.loads(line))
df_left = converter.process(data)

In [7]:
def hash_retrieve(df):
    """
    df : dataframe of tweets
    Description: 
        The function takes as an object a df of tweets obtained via twarc and 
        returns a generator object.
    
    """

    for line, id in zip(df['entities.hashtags'], df['id']):
        if pd.isna(line):
            continue
        line = line.strip()
        data = json.loads(line)
        for hashtag in ensure_flattened(data):
            yield [hashtag['tag'], id]

In [8]:
hashtags_df = pd.DataFrame(list(hash_retrieve(df_left)),
                 columns=['tweet_hashtags', 'id'])

hashtags_df = hashtags_df.groupby('id')['tweet_hashtags'].apply(lambda x: ','.join(x))

df_left = df_left.merge(hashtags_df, how='left', left_on='id', right_on='id')
df_left['tweet_hashtags']=df_left['tweet_hashtags'].str.lower()

In [9]:
hasht=[]
for i in df_left.index:
    try:
        hasht.append(df_left['tweet_hashtags'][i].split(","))
    except:
        hasht.append([])
    
    
df_left['list_hashtags']=hasht

In [10]:
hashtags=[]
for i in df_left.index:
    hashtags+=df_left['list_hashtags'][i]

**Manual revision:** Review the 100 most popular hashtags for left leaning politician during the selected period.

In [11]:
from collections import Counter
count = Counter(hashtags)
most_occur = count.most_common(100)
  
print(most_occur)

[('boricpresidente', 1095), ('seguimos', 606), ('boricpresidentedechile', 312), ('debateanatel', 260), ('cuartoretiro', 253), ('elecciones2021cl', 198), ('apruebodignidad', 195), ('franjaelectoral', 179), ('rutaesperanzaxboric', 176), ('matrimonioigualitario', 161), ('acusaciónconstitucional', 161), ('esposible', 154), ('boricnosune', 152), ('boricpresidente2022', 152), ('ladignidadesahora', 148), ('unmillondepuertasxboric', 142), ('1millondepuertasxboric', 137), ('meunoconboric', 136), ('chile', 131), ('votacomunista', 125), ('valdiviacl', 125), ('acusacionconstitucionalapiñera', 123), ('debatepresidencial2021', 121), ('ahorayasna', 117), ('boricpresidentedechile2022', 108), ('boricenprimeravuelta', 106), ('iquique', 106), ('coquimbo', 105), ('cuartoretirodel10xciento', 93), ('arica', 90), ('losrios', 88), ('vota1', 87), ('laserena', 82), ('paravivirmejor', 81), ('paraquechilesícambie', 68), ('covid19', 68), ('sueltenlasmicros', 66), ('atacama', 62), ('actitudregionalista', 62), ('deb

**Manual input** Decide the list of hashtags to label users. Think in hashtags that clearly identified the user with the left leaning. Don't use the # before the hashtag and use all the characters in lowercase. Max: 15 hashtags.

In [12]:
left_hashtags=["boricpresidente","seguimos", "boricpresidentedechile", "apruebodignidad", "rutaesperanzaxboric", "boricnosune", "boricpresidente2022", "unmillondepuertasxboric", "1millondepuertasxboric", "meunoconboric", "ahorayasna", "boricpresidentedechile2022", "boricenprimeravuelta", "vota1", "paravivirmejor"]

In [13]:
converter = DataFrameConverter()
data = []
with open(path_data+sep+"right_hashtags.json") as f:
    for line in f:
        data.append(json.loads(line))
df_right = converter.process(data)

In [14]:
hashtags_df = pd.DataFrame(list(hash_retrieve(df_right)),
                 columns=['tweet_hashtags', 'id'])

hashtags_df = hashtags_df.groupby('id')['tweet_hashtags'].apply(lambda x: ','.join(x))

df_right = df_right.merge(hashtags_df, how='left', left_on='id', right_on='id')

df_right['tweet_hashtags']=df_right['tweet_hashtags'].str.lower()

In [15]:
hasht=[]
for i in df_right.index:
    try:
        hasht.append(df_right['tweet_hashtags'][i].split(","))
    except:
        hasht.append([])
    
    
df_right['list_hashtags']=hasht

In [16]:
hashtags=[]
for i in df_right.index:
    hashtags+=df_right['list_hashtags'][i]

**Manual revision:** Review the 100 most popular hashtags for right leaning politician during the selected period.

In [17]:
from collections import Counter
count = Counter(hashtags)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = count.most_common(100)
  
print(most_occur)

[('covid_19', 406), ('sigamoscuidándonos', 304), ('vamosdefrente', 215), ('dosisderefuerzo', 207), ('yomevacuno', 204), ('atreveteconkast', 189), ('vanrysselberghesenadorbiobio', 186), ('kastpresidente2022', 166), ('valdiviacl', 152), ('kastpresidente', 139), ('enterreno', 111), ('siemprepresente', 108), ('valdivia', 102), ('laaraucanía', 100), ('leonidasromerodiputado', 100), ('vota2votakast', 90), ('elranco', 88), ('elecciones2021cl', 83), ('atreveteporchile', 76), ('antofagasta', 76), ('losrios', 74), ('distrito9', 69), ('sepuede', 68), ('todochileconkast', 68), ('losríos', 66), ('atréveteconkast', 64), ('covid', 63), ('fonotta', 63), ('chile', 62), ('votaaa60', 59), ('kastledaesperanzaachile', 57), ('asistencia100porciento', 57), ('lascondes', 56), ('iquique', 53), ('uniendoaladerecha', 53), ('vitacura', 50), ('lobarnechea', 50), ('lareina', 49), ('fiscalizaconmigo', 49), ('chilevotakast', 48), ('atrevidos', 47), ('covid19', 46), ('vota', 46), ('aa64', 46), ('debateanatel', 45), ('

**Manual input** Decide the list of hashtags to label users. Think in hashtags that clearly identified the user with the right leaning. Don't use the # before the hashtag and use all the characters in lowercase. Max: 15 hashtags.

In [18]:
right_hashtags=["atreveteconkast", "kastpresidente2022", "kastpresidente","vota2votakast", "atreveteporchile", "sepuede", "todochileconkast", "kastledaesperanzaachile", "chilevotakast", "atrevidos", "consichelsepuede", "sichelpresidente", "atrevidosporkast", "votakast","mujeresporkast"]

In [19]:
filename = path_data+sep+'left_hashtags_for_label.sav'
pickle.dump((left_hashtags), open(filename, 'wb'))

filename = path_data+sep+'right_hashtags_for_label.sav'
pickle.dump((right_hashtags), open(filename, 'wb'))